<a href="https://colab.research.google.com/github/LichenXia198/Deeping-Learning-project/blob/main/Reproducibility_project_source_code_Group_116.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup
from sklearn.cluster import MeanShift
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist 
!pip install POT
from scipy.io import loadmat
import ot
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
#preprocess datasets
#process amazon datset
amazon = loadmat('amazon_decaf.mat')
#Normalization
norm = np.linalg.norm(amazon["feas"])
amazon_feas = torch.from_numpy(amazon["feas"]/norm)
amazon_labels = torch.zeros(len(amazon["labels"]),10)
#convert label to one-hot key
for i in range(len(amazon["labels"])):
  amazon_labels[i][amazon["labels"][i][0]-1] = 1
#Ramdomize the order of samples
idx = torch.randperm(amazon_feas.shape[0])
amazon_feas = amazon_feas[idx].view(amazon_feas.size())
amazon_labels = amazon_labels[idx].view(amazon_labels.size())
#split dataset
train = math.ceil(amazon_feas.shape[0]*0.7)
vaild = math.floor(amazon_feas.shape[0]*0.2)
test = amazon_feas.shape[0]-train-vaild
amazon_feas, amazon_feas_v, amazon_feas_t = torch.split(amazon_feas, [train,vaild,test])
amazon_labels, amazon_labels_v, amazon_labels_t = torch.split(amazon_labels, [train,vaild,test])
#################

#process caltech datset
caltech = loadmat('caltech_decaf.mat')
#Normalization
norm = np.linalg.norm(caltech["feas"])
caltech_feas = torch.from_numpy(caltech["feas"]/norm)
caltech_labels = torch.zeros(len(caltech["labels"]),10)
#convert label to one-hot key
for i in range(len(caltech["labels"])):
  caltech_labels[i][caltech["labels"][i][0]-1] = 1
#Ramdomize the order of samples
idx = torch.randperm(caltech_feas.shape[0])
caltech_feas = caltech_feas[idx].view(caltech_feas.size())
caltech_labels = caltech_labels[idx].view(caltech_labels.size())
#split dataset
train = math.ceil(caltech_feas.shape[0]*0.7)
vaild = math.floor(caltech_feas.shape[0]*0.2)
test = caltech_feas.shape[0]-train-vaild
caltech_feas, caltech_feas_v, caltech_feas_t = torch.split(caltech_feas, [train,vaild,test])
caltech_labels, caltech_labels_v, caltech_labels_t = torch.split(caltech_labels, [train,vaild,test])

###############

#process dslr datset
dslr = loadmat('dslr_decaf.mat')
#Normalization
norm = np.linalg.norm(dslr["feas"])
dslr_feas = torch.from_numpy(dslr["feas"]/norm)
dslr_labels = torch.zeros(len(dslr["labels"]),10)
#convert label to one-hot key
for i in range(len(dslr["labels"])):
  dslr_labels[i][dslr["labels"][i][0]-1] = 1
#Ramdomize the order of samples
idx = torch.randperm(dslr_feas.shape[0])
dslr_feas = dslr_feas[idx].view(dslr_feas.size())
dslr_labels = dslr_labels[idx].view(dslr_labels.size())
#split dataset
train = math.ceil(dslr_feas.shape[0]*0.7)
vaild = math.floor(dslr_feas.shape[0]*0.2)
test = dslr_feas.shape[0]-train-vaild
dslr_feas, dslr_feas_v, dslr_feas_t = torch.split(dslr_feas, [train,vaild,test])
dslr_labels, dslr_labels_v, dslr_labels_t = torch.split(dslr_labels, [train,vaild,test])

###############

#process webcam datset
webcam = loadmat('webcam_decaf.mat')
#Normalization
norm = np.linalg.norm(webcam["feas"])
webcam_feas = torch.from_numpy(webcam["feas"]/norm)
webcam_labels = torch.zeros(len(webcam["labels"]),10)
#convert label to one-hot key
for i in range(len(webcam["labels"])):
  webcam_labels[i][webcam["labels"][i][0]-1] = 1
#Ramdomize the order of samples
idx = torch.randperm(webcam_feas.shape[0])
webcam_feas = webcam_feas[idx].view(webcam_feas.size())
webcam_labels = webcam_labels[idx].view(webcam_labels.size())
#split dataset
train = math.ceil(webcam_feas.shape[0]*0.7)
vaild = math.floor(webcam_feas.shape[0]*0.2)
test = webcam_feas.shape[0]-train-vaild
webcam_feas, webcam_feas_v, webcam_feas_t = torch.split(webcam_feas, [train,vaild,test])
webcam_labels, webcam_labels_v, webcam_labels_t = torch.split(webcam_labels, [train,vaild,test])

In [ ]:
# combine sourse datasets
all_features = torch.cat((webcam_feas,amazon_feas,dslr_feas))
all_labels = torch.cat((webcam_labels,amazon_labels,dslr_labels))
print(all_features.shape)
print(all_labels.shape)

In [ ]:
#One layer network (classifer needs to be optimized)
class MyNet(nn.Module):
    
    
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(4096, 10)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        return h



In [ ]:
# Tensorboard
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

#Additional Setup to use Tensorboard
!pip install -q tensorflow
%load_ext tensorboard
# Open Tensorboard
%tensorboard --logdir runs/

In [ ]:
writer = SummaryWriter()

# Calculate Wasserste Distance
def my_loss_custom(beta,G,C0,target_predict,y):
  loss = torch.sum(torch.square(torch.cdist(y,target_predict))*G)
  C = beta*C0 + torch.square(torch.cdist(y,target_predict))
  lose = C*G
  return torch.sum(lose)

# train and test JDOT with test set/validation set
def train_jodt(le,wd,all_features,target,target_t,all_labels,target_labels,target_labels_t,writer):
  n1 = list(all_features.size())[0]
  ntest =list(target.size())[0]
  wa1=np.ones((n1,))/n1
  wb=np.ones((ntest,))/ntest 
  y1 = all_labels
  orign = target_labels
  orign_test = target_labels_t
  target = target.float()
  target_test=target_t
  target_test = target_test.float()
  all_features = all_features.float()
  all_labels = all_labels.float()

  C1=torch.square(torch.cdist(all_features,target))
  beta1 = 1/torch.max(C1).data.item()
  C1=C1/torch.max(C1)

  f = MyNet()

  optimizer = optim.Adam(f.parameters(),lr=le,betas=(0.9,0.99),weight_decay=wd ,eps=math.exp(-8))
  
  ave_train_loss = 0
  ave_test_loss = 0
  ave_train_acc = 0
  ave_test_acc = 0
  convergence = 0
  previous_loss = torch.zeros(1)
  k = 0

  while True:
  
    tp = f(target)
    
    # Calculte Optimal Transport matrix
    C_1 = beta1*C1+ torch.square((torch.cdist(y1,tp)))
    G1_solution=ot.emd(wa1,wb,C_1.detach().numpy())
    G1=torch.from_numpy(G1_solution)
  
    optimizer.zero_grad()
    # Calculate Wasserste Distance
    lose1 = my_loss_custom(beta1,G1,C1,tp,y1)
    lose = lose1
    lose.backward()
    optimizer.step()

    #calculate test lose test accurcy, train lose and train accurcy
    trainloss = torch.mean((tp - orign) ** 2)
    traincorrect = torch.sum(torch.argmax(tp, axis=1) == torch.argmax(orign, axis=1))/len(tp)

    test_predict = f(target_test)
    testloss = torch.mean((test_predict - orign_test) ** 2)
    testcorrect = torch.sum(torch.argmax(test_predict, axis=1) == torch.argmax(orign_test, axis=1))/len(test_predict)

    ave_train_loss += trainloss
    ave_train_acc += traincorrect.item()

    ave_test_loss += testloss
    ave_test_acc += testcorrect.item()
  
    # print test lose test accurcy, train lose and train accurcy in tensorboard
    if k%10 ==0 :
      train_loss = ave_train_loss/10
      test_loss = ave_test_loss/10
      writer.add_scalars('Loss', {'Train': train_loss, 'Test':test_loss}, k)
      ave_train_loss = 0
      ave_test_loss = 0
      train_acc = ave_train_acc/10
      test_acc = ave_test_acc/10
      writer.add_scalars('Accuracy',{'Train': train_acc,'Test':test_acc} , k)
      ave_train_acc = 0
      ave_test_acc = 0
    
    # Check if convergent
    if k%10 ==0:
      if previous_loss.data.item() -  lose.data.item() <= 0.0001:
        convergence+=1
      else:
        convergence = 0
      previous_loss = lose.detach().clone()
    if convergence>5:
      break
    k+=1



  # Calculate final test accurcy and print
  test_predict = f(target_test)
  correct = torch.argmax(test_predict, axis=1) == torch.argmax(orign_test, axis=1)
  print("jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj")
  print("convergent!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
  print(torch.sum(correct)/len(test_predict))
  print("jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj")

In [ ]:
# an example for training and testing JDOT
train_jodt(0.06,0.0002,all_features,caltech_feas,caltech_feas_t,all_labels,caltech_labels,caltech_labels_t,writer)

In [ ]:
writer = SummaryWriter()

# Nomalize alphas
def normalize_alphas_inplace(alphas):
  alpha1, alpha2, alpha3 = alphas
  alpha1 = torch.clamp_(alpha1, min=0)
  alpha2 = torch.clamp_(alpha2, min=0)
  alpha3 = torch.clamp_(alpha3, min=0)
  sum = alpha1+alpha2+alpha3
  alpha1 = alpha1.div_(sum)
  alpha2 = alpha2.div_(sum)
  alpha3 = alpha3.div_(sum)
  return alpha1, alpha2, alpha3



# train and test WJDOT with test set/validation set
def train(lerate,wd,lra,source1,source2,source3,target,source1_labels,source2_labels,source3_labels,target_labels,target_labels_t,target_t,writer):
  
  #Prepare datasets
  source1 = source1.float()
  source2 = source2.float()
  source3 = source3.float()
  target = target.float()
  n1 = list(source1.size())[0]
  n2 = list(source2.size())[0]
  n3 = list(source3.size())[0]
  ntest =list(target.size())[0]
  wa1=np.ones((n1,))/n1
  wa2=np.ones((n2,))/n2
  wa3=np.ones((n3,))/n3
  wb=np.ones((ntest,))/ntest 
  y1 = source1_labels
  y2 = source2_labels
  y3 = source3_labels
  orign = target_labels
  orign_test = target_labels_t

  target_test=target_t
  target_test = target_test.float()

  # Initialize alphas
  alpha1 = torch.ones(1)/3
  alpha2 = torch.ones(1)/3
  alpha3 = torch.ones(1)/3
  alpha1.requires_grad = True
  alpha2.requires_grad = True
  alpha3.requires_grad = True

  # Initialize embedding discrepancies and beta
  C1=torch.square(torch.cdist(source1,target))
  beta1 = 1/torch.max(C1).data.item()
  C1=C1/torch.max(C1)

  C2=torch.square(torch.cdist(source2,target))
  beta2 = 1/torch.max(C2).data.item()
  C2=C2/torch.max(C2)

  C3=torch.square(torch.cdist(source3,target))
  beta3 = 1/torch.max(C3).data.item()
  C3=C3/torch.max(C3)
  beta_b = (beta1+beta2+beta3)/3


  # Initialize model
  f = MyNet()

  optimizer = optim.Adam(f.parameters(),lr=lerate,betas=(0.9,0.99),weight_decay = wd,eps=math.exp(-8))
  optimizerA = optim.Adam([alpha1, alpha2, alpha3],lr=lra,betas=(0.9,0.99),eps=math.exp(-8))

  convergence = 0
  ave_train_loss = 0
  ave_test_loss = 0
  ave_train_acc = 0
  ave_test_acc = 0
  previous_loss = torch.zeros(1)
  k = 0

  # Start training
  while True:
  
    tp = f(target)
  
    # Normalize alphas
    with torch.no_grad():
      alpha1, alpha2, alpha3 = normalize_alphas_inplace((alpha1, alpha2, alpha3))
    
    # Overall Beta
    beta_b = alpha1*beta1 + alpha2*beta2 + alpha3*beta3

    # Optimal transport matrix for each source domain with target domain
    C_1 = beta_b*C1+ torch.square((torch.cdist(y1,tp)))
    G1_solution=ot.emd(wa1,wb,C_1.detach().numpy())
    G1=torch.from_numpy(G1_solution)


    C_2 = beta_b*C2+ torch.square((torch.cdist(y2,tp)))  
    G2_solution=ot.emd(wa2,wb,C_2.detach().numpy())
    G2=torch.from_numpy(G2_solution)
  

    C_3 = beta_b*C3+ torch.square((torch.cdist(y3,tp)))  
    G3_solution=ot.emd(wa3,wb,C_3.detach().numpy())
    G3=torch.from_numpy(G3_solution)

    optimizer.zero_grad()
    optimizerA.zero_grad()

    # Wasserste Distance for each source domain
    lose1 = my_loss_custom(beta_b,G1,C1,tp,y1)
    lose2 = my_loss_custom(beta_b,G2,C2,tp,y2)
    lose3 = my_loss_custom(beta_b,G3,C3,tp,y3)

    # Overall Wasserste Distance
    lose = (alpha1*lose1 + alpha2*lose2+ alpha3*lose3)

    lose.backward()
    optimizer.step()
    optimizerA.step()
    if k%10 == 0:
      # every 10 iterations check if convergent
      if previous_loss.data.item() -  lose.data.item() <= 0.001:
        convergence+=1
      else:
        convergence = 0
      previous_loss = lose.detach().clone()

      tp_t = tp.detach().clone()
      y1_t = y1.detach().clone()
      y2_t = y2.detach().clone()
      y3_t = y3.detach().clone()
      l1_t = torch.sum((1)*(torch.square((torch.cdist(y1_t,tp_t))))*(G1.detach().clone()))
      l2_t = torch.sum((1)*(torch.square((torch.cdist(y2_t,tp_t))))*(G2.detach().clone()))
      l3_t = torch.sum((1)*(torch.square((torch.cdist(y3_t,tp_t))))*(G3.detach().clone()))
      C1_t = torch.sum(beta_b*(C1.detach().clone())*(G1.detach().clone()))
      C2_t = torch.sum(beta_b*(C2.detach().clone())*(G2.detach().clone()))
      C3_t = torch.sum(beta_b*(C3.detach().clone())*(G3.detach().clone()))

      #print Wasserste Distance
      print("1: ")
      print(C1_t+l1_t)
      print("2: ")
      print(C2_t+l2_t)
      print("3: ")
      print(C3_t+l3_t)
      print("##############")
      #print Alpha
      print("111")
      print(alpha1)
      print("222")
      print(alpha2)
      print("333")
      print(alpha3)
      print("+++++++++++++++++++++")

    #calculate test lose test accurcy, train lose and train accurcy    
    trainloss = torch.mean((tp - orign) ** 2)
    traincorrect = torch.sum(torch.argmax(tp, axis=1) == torch.argmax(orign, axis=1))/len(tp)

    test_predict = f(target_test)
    testloss = torch.mean((test_predict - orign_test) ** 2)
    testcorrect = torch.sum(torch.argmax(test_predict, axis=1) == torch.argmax(orign_test, axis=1))/len(test_predict)

    ave_train_loss += trainloss
    ave_train_acc += traincorrect.item()

    ave_test_loss += testloss
    ave_test_acc += testcorrect.item()
  
    # print test lose test accurcy, train lose and train accurcy in tensorboard
    if k%10 ==0 :
      train_loss = ave_train_loss/10
      test_loss = ave_test_loss/10
      writer.add_scalars('WLoss', {'Train': train_loss, 'Test':test_loss}, k)
      writer.add_scalars('WassersteinLoss', {'Train': lose}, k)
      ave_train_loss = 0
      ave_test_loss = 0
      train_acc = ave_train_acc/10
      test_acc = ave_test_acc/10
      writer.add_scalars('WAccuracy',{'Train': train_acc,'Test':test_acc} , k)
      ave_train_acc = 0
      ave_test_acc = 0

    if convergence>5:
      break
    k+=1


  # Calculate final test accurcy and print
  test_predict = f(target_test)
  testloss = torch.mean((test_predict - orign_test) ** 2)
  correct = torch.argmax(test_predict, axis=1) == torch.argmax(orign_test, axis=1)
  print("############################")
  print("convergent!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
  print(torch.sum(correct)/len(test_predict))
  print("############################")

In [ ]:
# an example for training and testing WJDOT
train(0.06,0.0002,0.001,amazon_feas,dslr_feas,caltech_feas,webcam_feas,amazon_labels,dslr_labels,caltech_labels,webcam_labels,webcam_labels_t,webcam_feas_t,writer)

In [ ]:
# a demonstration of tune hyperparameters
lerates = [0.03,0.06,0.09,0.001,0.003,0.006,0.009,0.0001,0.0003,0.0006,0.0009,0.4,0.8,0.04,0.08,0.004,0.008,0.0004,0.0008]
wds = [0.03,0.06,0.09,0.001,0.003,0.006,0.009,0.0001,0.0003,0.0006,0.0009,0.4,0.8,0.04,0.08,0.004,0.008,0.0004,0.0008]
ass = [0.01,0.03,0.06,0.09,0.001,0.003,0.006,0.009,0.0001,0.0003,0.0006,0.0009,0.4,0.8,0.04,0.08,0.004,0.008,0.0004,0.0008]
for wd in wds:
  for le in lerates:
    for a in ass:
    #train_jodt(le,wd,all_features,webcam_feas,webcam_feas_v,all_labels,webcam_labels,webcam_labels_v,writer)
      train(le,wd,a,amazon_feas,dslr_feas,caltech_feas,webcam_feas,amazon_labels,dslr_labels,caltech_labels,webcam_labels,webcam_labels_v,webcam_feas_v,writer)
    